<h1> Довільний текст <h1>

Підрахунок частот букв і частот біграм в тексті що перетинаються, та  що не перетинаються

In [16]:
from collections import Counter
import re
import chardet
import pandas as pd

# Функція для підготовки тексту
def prepare_text(text):
    # Перетворюємо текст на нижній регістр, замінюємо символи на пробіли
    text = re.sub(r'[^a-zA-Zа-яА-Я]', ' ', text.lower())
    # Замінюємо послідовності пробілів на один пробіл
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Функція для підрахунку частоти букв
def letter_frequency(text):
    freq = Counter(text)  
    total_letters = sum(freq.values())  
    freq_percent = {letter: count / total_letters for letter, count in freq.items()}
    return dict(sorted(freq_percent.items(), key=lambda item: item[1], reverse=True))

# Функція для підрахунку частоти біграм(з перетином та без)
def bigram_frequency(text):
    # Біграми з перетином
    bigrams_with_overlap = [text[i:i+2] for i in range(len(text) - 1)]
    total_bigrams_with_overlap = len(bigrams_with_overlap) 
    freq_with_overlap = Counter(bigrams_with_overlap)

    # Біграми без перетину
    bigrams_without_overlap = [text[i:i+2] for i in range(0, len(text) - 1, 2)]
    total_bigrams_without_overlap = len(bigrams_without_overlap)  
    freq_without_overlap = Counter(bigrams_without_overlap)

    # Перетворюємо частоти у відсоткові значення
    freq_with_overlap = {bigram: count / total_bigrams_with_overlap for bigram, count in freq_with_overlap.items()}
    freq_without_overlap = {bigram: count / total_bigrams_without_overlap for bigram, count in freq_without_overlap.items()}

    return freq_with_overlap, freq_without_overlap

# Визначаємо кодування файлу
with open('./text2', 'rb') as file:
    raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding'] if result['encoding'] else 'utf-8'
    print(f"Виявлене кодування: {encoding}\n")

with open('./text2', 'r', encoding=encoding) as file:
    text = file.read()

prepared_text = prepare_text(text)

letter_freq = letter_frequency(prepared_text)
bigram_freq_with_overlap, bigram_freq_without_overlap = bigram_frequency(prepared_text)

# Зберігаємо в один Excel файл
with pd.ExcelWriter("frequency_data.xlsx") as writer:
    # частота букв
    letter_df = pd.DataFrame(list(letter_freq.items()), columns=["Буква", "Частота"])
    letter_df.to_excel(writer, sheet_name="Частота Букв", index=False)

    # частота біграм з перетином
    bigram_with_overlap_df = pd.DataFrame(list(bigram_freq_with_overlap.items()), columns=["Біграма", "Частота"])
    bigram_with_overlap_df.to_excel(writer, sheet_name="Частота Біграм З Перетином", index=False)

    # частота біграм без перетину
    bigram_without_overlap_df = pd.DataFrame(list(bigram_freq_without_overlap.items()), columns=["Біграма", "Частота"])
    bigram_without_overlap_df.to_excel(writer, sheet_name="Частота Біграм Без Перетину", index=False)

print("\nЧастота букв і біграм збережена у файл frequency_data.xlsx")


Виявлене кодування: UTF-8-SIG


Частота букв і біграм збережена у файл frequency_data.xlsx


Обчислення ентропії та надлишковості тексту 

In [17]:
import math
from collections import Counter

def calc_entropy(ngrams, total_count):
    entropy = 0
    for count in ngrams.values():
        probability = count / total_count
        entropy -= probability * math.log2(probability)
    return entropy

def get_ngrams(text, n, overlap=True):
    if overlap:
        return [text[i:i+n] for i in range(len(text) - n + 1)]
    else:
        return [text[i:i+n] for i in range(0, len(text) - n + 1, n)]

def read_file(file_path):
    # Читання файлу
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
    except UnicodeDecodeError:
        print("Не вдалося прочитати файл у кодуванні UTF-8. Спроба з іншим кодуванням...")
        with open(file_path, 'r', encoding='windows-1252', errors='replace') as file:
            text = file.read()
    return text  

def calc_redundancy(H, N, is_bigram=False):
    # Обчислення максимальної ентропії H_max для монограм і біграм
    if is_bigram:
        H_max = math.log2(N ** 2)  # Максимальна ентропія для біграм
    else:
        H_max = math.log2(N)  # Максимальна ентропія для монограм
    
    # Обчислення надлишковості R
    R = 1 - (H / H_max)
    return R

def main(file_path):
    text = read_file(file_path)
    
     
    # Підрахунок загальної кількості символів у тексті для визначення розміру алфавіту
    unique_chars = set(text)
    N = len(unique_chars)
    
    # Монограми (H1)
    monograms = get_ngrams(text, 1)
    if len(monograms) > 0:
        monogram_counts = Counter(monograms)
        H1 = calc_entropy(monogram_counts, len(monograms))
        R1 = calc_redundancy(H1, N, is_bigram=False)
        print(f"Ентропія H1 (монограми): {H1:.5f}")
        print(f"Надлишковість R1 (монограми): {R1:.5f}")
    
    # Біграми з перетином (H2)
    bigrams_with_overlap = get_ngrams(text, 2, overlap=True)
    if len(bigrams_with_overlap) > 0:
        bigram_counts_with_overlap = Counter(bigrams_with_overlap)
        H2_with_overlap = calc_entropy(bigram_counts_with_overlap, len(bigrams_with_overlap))
        R2_with_overlap = calc_redundancy(H2_with_overlap, N, is_bigram=True)
        print(f"Ентропія H2 (біграми з перетином): {H2_with_overlap:.5f}")
        print(f"Надлишковість R2 (біграми з перетином): {R2_with_overlap:.5f}")
    
    # Біграми без перетину (H2 без перетину)
    bigrams_without_overlap = get_ngrams(text, 2, overlap=False)
    if len(bigrams_without_overlap) > 0:
        bigram_counts_without_overlap = Counter(bigrams_without_overlap)
        H2_without_overlap = calc_entropy(bigram_counts_without_overlap, len(bigrams_without_overlap))
        R2_without_overlap = calc_redundancy(H2_without_overlap, N, is_bigram=True)
        print(f"Ентропія H2 (біграми без перетину): {H2_without_overlap:.5f}")
        print(f"Надлишковість R2 (біграми без перетину): {R2_without_overlap:.5f}")

# Виклик функції main із шляхом до файлу
file_path = "./text2"
main(file_path)


Ентропія H1 (монограми): 4.61260
Надлишковість R1 (монограми): 0.23961
Ентропія H2 (біграми з перетином): 8.10212
Надлишковість R2 (біграми з перетином): 0.33218
Ентропія H2 (біграми без перетину): 8.10161
Надлишковість R2 (біграми без перетину): 0.33222


<h1> Текст без пробілів <h1>

 Видалення пробілів із тексту

In [18]:
def remove_spaces(input_file, output_file):
    try:
        with open(input_file, 'r', encoding='utf-8') as infile:
            text = infile.read()

        #Видаляємо всі пробіли
        text_without_spaces = text.replace(" ", "")

        #Записуємо текст у новий файл
        with open(output_file, 'w', encoding='utf-8') as outfile:
            outfile.write(text_without_spaces)

        print(f"Файл '{output_file}' успішно створено.")
    except FileNotFoundError:
        print(f"Файл '{input_file}' не знайдено.")
    except Exception as e:
        print(f"Сталася помилка: {e}")

input_file = './text2'  #вхідний файл
output_file = './text3'  #вихідний файл
remove_spaces(input_file, output_file)


Файл './text3' успішно створено.


Застосування попередніх програм до text3 - текст без пробілів 

In [19]:
# Функція для підрахунку частоти букв
def letter_frequency(text):
    freq = Counter(text)  
    total_letters = sum(freq.values())  
    freq_percent = {letter: count / total_letters for letter, count in freq.items()}
    return dict(sorted(freq_percent.items(), key=lambda item: item[1], reverse=True))

# Функція для підрахунку частоти біграм(з перетином та без)
def bigram_frequency(text):
    # Біграми з перетином
    bigrams_with_overlap = [text[i:i+2] for i in range(len(text) - 1)]
    total_bigrams_with_overlap = len(bigrams_with_overlap) 
    freq_with_overlap = Counter(bigrams_with_overlap)

    # Біграми без перетину
    bigrams_without_overlap = [text[i:i+2] for i in range(0, len(text) - 1, 2)]
    total_bigrams_without_overlap = len(bigrams_without_overlap)  
    freq_without_overlap = Counter(bigrams_without_overlap)

    # Перетворюємо частоти у відсоткові значення
    freq_with_overlap = {bigram: count / total_bigrams_with_overlap for bigram, count in freq_with_overlap.items()}
    freq_without_overlap = {bigram: count / total_bigrams_without_overlap for bigram, count in freq_without_overlap.items()}

    return freq_with_overlap, freq_without_overlap

# Визначаємо кодування файлу
with open('./text2', 'rb') as file:
    raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding'] if result['encoding'] else 'utf-8'
    print(f"Виявлене кодування: {encoding}\n")

with open('./text3', 'r', encoding=encoding) as file:
    text = file.read()

prepared_text = prepare_text(text)

letter_freq = letter_frequency(prepared_text)
bigram_freq_with_overlap, bigram_freq_without_overlap = bigram_frequency(prepared_text)

# Зберігаємо в один Excel файл
with pd.ExcelWriter("frequency_data_spaces_del.xlsx") as writer:
    # частота букв
    letter_df = pd.DataFrame(list(letter_freq.items()), columns=["Буква", "Частота"])
    letter_df.to_excel(writer, sheet_name="Частота Букв", index=False)

    # частота біграм з перетином
    bigram_with_overlap_df = pd.DataFrame(list(bigram_freq_with_overlap.items()), columns=["Біграма", "Частота"])
    bigram_with_overlap_df.to_excel(writer, sheet_name="Частота Біграм З Перетином", index=False)

    # частота біграм без перетину
    bigram_without_overlap_df = pd.DataFrame(list(bigram_freq_without_overlap.items()), columns=["Біграма", "Частота"])
    bigram_without_overlap_df.to_excel(writer, sheet_name="Частота Біграм Без Перетину", index=False)

print("\nЧастота букв і біграм збережена у файл frequency_data_spaces_del.xlsx")


Виявлене кодування: UTF-8-SIG


Частота букв і біграм збережена у файл frequency_data_spaces_del.xlsx


In [20]:
def calc_entropy(ngrams, total_count):
    entropy = 0
    for count in ngrams.values():
        probability = count / total_count
        entropy -= probability * math.log2(probability)
    return entropy

def get_ngrams(text, n, overlap=True):
    if overlap:
        return [text[i:i+n] for i in range(len(text) - n + 1)]
    else:
        return [text[i:i+n] for i in range(0, len(text) - n + 1, n)]

def read_file(file_path):
    # Читання файлу
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
    except UnicodeDecodeError:
        print("Не вдалося прочитати файл у кодуванні UTF-8. Спроба з іншим кодуванням...")
        with open(file_path, 'r', encoding='windows-1252', errors='replace') as file:
            text = file.read()
    return text  

def calc_redundancy(H, N, is_bigram=False):
    # Обчислення максимальної ентропії H_max для монограм і біграм
    if is_bigram:
        H_max = math.log2(N ** 2)  # Максимальна ентропія для біграм
    else:
        H_max = math.log2(N)  # Максимальна ентропія для монограм
    
    # Обчислення надлишковості R
    R = 1 - (H / H_max)
    return R

def main(file_path):
    text = read_file(file_path)
    
     
    # Підрахунок загальної кількості символів у тексті для визначення розміру алфавіту
    unique_chars = set(text)
    N = len(unique_chars)
    
    # Монограми (H1)
    monograms = get_ngrams(text, 1)
    if len(monograms) > 0:
        monogram_counts = Counter(monograms)
        H1 = calc_entropy(monogram_counts, len(monograms))
        R1 = calc_redundancy(H1, N, is_bigram=False)
        print(f"Ентропія H1 (монограми): {H1:.5f}")
        print(f"Надлишковість R1 (монограми): {R1:.5f}")
    
    # Біграми з перетином (H2)
    bigrams_with_overlap = get_ngrams(text, 2, overlap=True)
    if len(bigrams_with_overlap) > 0:
        bigram_counts_with_overlap = Counter(bigrams_with_overlap)
        H2_with_overlap = calc_entropy(bigram_counts_with_overlap, len(bigrams_with_overlap))
        R2_with_overlap = calc_redundancy(H2_with_overlap, N, is_bigram=True)
        print(f"Ентропія H2 (біграми з перетином): {H2_with_overlap:.5f}")
        print(f"Надлишковість R2 (біграми з перетином): {R2_with_overlap:.5f}")
    
    # Біграми без перетину (H2 без перетину)
    bigrams_without_overlap = get_ngrams(text, 2, overlap=False)
    if len(bigrams_without_overlap) > 0:
        bigram_counts_without_overlap = Counter(bigrams_without_overlap)
        H2_without_overlap = calc_entropy(bigram_counts_without_overlap, len(bigrams_without_overlap))
        R2_without_overlap = calc_redundancy(H2_without_overlap, N, is_bigram=True)
        print(f"Ентропія H2 (біграми без перетину): {H2_without_overlap:.5f}")
        print(f"Надлишковість R2 (біграми без перетину): {R2_without_overlap:.5f}")

# Виклик функції main із шляхом до файлу
file_path = "./text3"
main(file_path)


Ентропія H1 (монограми): 4.63480
Надлишковість R1 (монограми): 0.23321
Ентропія H2 (біграми з перетином): 8.26551
Надлишковість R2 (біграми з перетином): 0.31627
Ентропія H2 (біграми без перетину): 8.26566
Надлишковість R2 (біграми без перетину): 0.31625
